### Imports

In [1]:
from keras.layers import Dense
from keras.layers.recurrent import GRU
from keras.models import Sequential

from tools import *
import string

Using TensorFlow backend.


### Configure Letters Range and Encryption Function

In [2]:
set_characters(string.ascii_uppercase)
encrypt = caesar_cipher

### Generating and Preparing Data

In [3]:
text = generate_text(length=5, limit=1000000)

train_cipher = to_vec(map(encrypt, text))
train_text = to_vec(text)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=train_cipher.shape[1:]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=1)

Train on 900000 samples, validate on 100000 samples
Epoch 1/1
900000/900000 [==============================] - 75s 83us/step - loss: 8.1829 - acc: 0.6558 - val_loss: 1.3380e-10 - val_acc: 1.0000


### Predicting and Evaluating the Model

In [5]:
test_cyhper = 'ABXYZ'
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: XYUVW
Test cipher vs encrypted prediction: ABXYZ vs ABXYZ
